<a href="https://colab.research.google.com/github/iamnarendran/Business-FAQ-Chatbot-RAG/blob/main/Business_FAQ_Chatbot_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


> # **Business FAQ Chatbot (RAG-based)**

In [1]:
# Installing necessary libraries
!pip install -q chromadb
!pip install -q sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.0 MB/s eta 

In [52]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions

# Making the document chunks (Q&A style)
sample_text = """

Q1: What services do you provide?
A: We specialize in Business Intelligence, Data Science, Machine Learning, and Conversational AI solutions using Generative AI models.

Q2: Who are your clients?
A: Our clients include enterprises in retail, healthcare, finance, and logistics who are looking to automate insights and decision-making using AI.

Q3: What technologies do you use?
A: We utilize tools like Python, Google Cloud, TensorFlow, and various open-source LLMs including fine-tuned conversational agents.

Q4: Do you provide chatbot solutions?
A: Yes, we build intelligent chatbots using LLMs that integrate with CRMs, FAQs, internal documentation, and support systems.

Q5: Where is your team located?
A: Our headquarters is in Chennai, India, and we operate with clients globally via remote infrastructure.

Q6: Can you build domain-specific LLM applications?
A: Absolutely! We customize LLM-powered tools for retail insights, HR automation, financial report summarization, and more.

Q7: Do you offer internships?
A: Yes, we provide internships in AI/ML, LLM engineering, and data analysis — focused on practical, hands-on work.

Q8: What industries do you specialize in?
A: We work across diverse domains such as e-commerce, finance, education, healthcare, and logistics, delivering custom AI-powered solutions.

Q9: How do you ensure data security in your AI applications?
A: We follow industry best practices including data anonymization, secure API layers, role-based access, and compliance with GDPR/ISO standards.

Q10: Can your solutions integrate with existing systems?
A: Yes, we build flexible APIs and connectors that can integrate seamlessly with CRMs, ERPs, cloud storage, and existing enterprise infrastructure.

Q11: What is your typical project timeline?
A: Timelines vary, but POCs usually take 2–4 weeks, while full deployments range from 6–12 weeks based on scope and complexity.

Q12: Do you offer post-deployment support?
A: Absolutely. We offer ongoing maintenance, monitoring, model retraining, and performance optimization based on client needs.

Q13: What kind of GenAI projects have you delivered?
A: We've built text summarizers, business chatbots, resume reviewers, legal AI assistants, and multilingual support tools using open-source LLMs.

Q14: Can your chatbots support multiple languages?
A: Yes, our LLM-based assistants can be configured to understand and respond in English, Hindi, Tamil, and several other regional or global languages.

Q15: Do you offer training or workshops for internal teams?
A: Yes, we provide onboarding sessions, AI upskilling programs, and documentation to help your teams leverage AI solutions effectively.

Q16: Can you deploy solutions on-premise?
A: Yes, depending on client preference, we support cloud, hybrid, and fully on-prem deployments to ensure data locality and compliance.

Q17: How do you price your services?
A: Our pricing depends on project size, complexity, and engagement model (e.g., fixed, milestone-based, or retainer). We offer free consultations to scope projects.

"""

# Spliting into Q&A chunks
chunks = [chunk.strip() for chunk in sample_text.split("Q") if chunk.strip()]
chunks = [("Q" + chunk).strip() for chunk in chunks]  # Add back the 'Q'

# Setup ChromaDB client (in-memory)
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
collection = chroma_client.create_collection(name="faq_collection1")

# Loading embedding model
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# Adding embedded chunks to collection
embeddings = embed_model.encode(chunks).tolist()

collection.add(
    documents=chunks,
    embeddings=embeddings,
    ids=[f"doc{i}" for i in range(len(chunks))]
)

print("FAQ chunks embedded and stored successfully!")

FAQ chunks embedded and stored successfully!


In [42]:
#Checking how our embeddings work on questions
question = "Who are your clients?"
top_k=3
question_embedding = embed_model.encode([question]).tolist()[0]

print("Question emdedding (Retrived questions) \n")
print(question_embedding)

results = collection.query(query_embeddings=[question_embedding],n_results=top_k)

print("Retrived FAQ's \n")
results['documents'][0]

Question emdedding (Retrived questions) 

[-0.02957608550786972, 0.04186391085386276, -0.0087010208517313, -0.024030234664678574, -0.055884819477796555, 0.021520031616091728, 0.06888774037361145, -0.029949473217129707, 0.06706734746694565, -0.06529893726110458, -0.02326003834605217, 0.047076765447854996, 0.019597796723246574, 0.029591182246804237, 0.015474643558263779, 0.015786580741405487, 0.061644092202186584, -0.004915796220302582, 0.05554228648543358, -0.046077486127614975, -0.2002246230840683, -0.014997757971286774, -0.04913756623864174, 0.0034165263641625643, 0.015971951186656952, -0.04954485967755318, 0.03858521953225136, -0.012993848882615566, -0.06404611468315125, -0.04375194385647774, -0.057153962552547455, -0.02570831961929798, 0.02008519135415554, 0.02426292933523655, 0.02012987993657589, 0.0771365612745285, -0.03870512917637825, 0.03261876478791237, 0.006563838571310043, 0.03054017946124077, 0.02378295548260212, -0.002520170994102955, -0.011140587739646435, 0.0034525468945

['Q2: Who are your clients?\nA: Our clients include enterprises in retail, healthcare, finance, and logistics who are looking to automate insights and decision-making using AI.',
 'Q17: How do you price your services?\nA: Our pricing depends on project size, complexity, and engagement model (e.g., fixed, milestone-based, or retainer). We offer free consultations to scope projects.',
 'Q5: Where is your team located?\nA: Our headquarters is in Chennai, India, and we operate with clients globally via remote infrastructure.']

In [49]:
from openai import OpenAI

def get_relevant_chunks(question, top_k=3):
    # Embed the question
    question_embedding = embed_model.encode([question]).tolist()[0]

    # Query the ChromaDB collection
    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=top_k
    )

    return results['documents'][0]  # returns a list of top-k relevant chunks

def generate_answer_with_context(question):
    context_chunks = get_relevant_chunks(question, top_k=3)
    context = "\n".join(context_chunks)

    prompt = f"""You are a helpful AI assistant answering questions based on a company's FAQ, answer only based on the context,
    if the question is not in context, make response with the question they asked 'I dont have what you asked [question],please feel free to contact our mail',
    any answer is no more than 50 words. finally say 'I'm a chatbot powered by Deepseek AI'

Context:
{context}

Question: {question}
Answer:"""

    client = OpenAI(
    base_url="https://api.a4f.co/v1",
    api_key="A4F-KEYS",)  # You can get this API Keys for free from "https://www.a4f.co/models" and directly paste it.

    response = client.chat.completions.create(
        model="provider-3/deepseek-v3",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
    )

    return response.choices[0].message.content

In [50]:
question = "Do you offer internships?"
question1 = "Where is your office?"
question2 = "how many workers in your office?"
question4 = "How do you price your services?"

# Here we have asked two questions in same prompt
question3 = "Do you provide chatbot solutions & Can you build domain-specific LLM applications"
answer = generate_answer_with_context(question3)
print("💬 Answer:", answer)

💬 Answer: Yes, we provide intelligent chatbot solutions and can build domain-specific LLM applications tailored to industries like retail, HR, and finance. I'm a chatbot powered by Deepseek AI.
